##### Copyright 2017 Google LLC.

##### Modifications Copyright 2019 Tomoaki Masuda.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

このノートブックは、以下のノートブックを元に日本語訳、一部章立ての再構成、加筆を行いました。
https://colab.research.google.com/notebooks/magenta/music_vae/music_vae.ipynb


#  MusicVAE: 音楽の長期構造学習のための階層的潜在ベクトルモデル

### ___Adam Roberts, Jesse Engel, Colin Raffel, Curtis Hawthorne, and Douglas Eck___

 [MusicVAE](https://g.co/magenta/music-vae)は楽譜の潜在空間を学習し、さまざまなモードを提供します

- インタラクティブな音楽制作
- 事前分布からの無作為抽出
- 既存のシーケンス間の補間
- 属性ベクトルを介した既存のシーケンスの操作

これらの相互作用の例は以下で実際に試せます。また、作例は、[YouTubeのプレイリスト](https://www.youtube.com/playlist?list=PLBUMAYA6kvGU8Cgqh709o5SUvo-zHGTxr)で聴くことができます。

短いシーケンス（2小節の「ループ」など）は、双方向LSTMエンコーダを使用します。 より長いシーケンスは、新しく提案された階層的LSTMを使います。これは、モデルがより長い構造を学習するのに役立ちます。 

また、複数のデコーダを、階層デコーダの最下層のembeddingで学習させることで、楽器間の相互依存関係をモデル化します。

[ブログ記事](https://g.co/magenta/music-vae)と[論文](https://goo.gl/magenta/musicvae-paper)に、より詳細の情報があります。 

---

このノートブックは自己完結型であり、Googleクラウドでネイティブに動作します。

独自のモデルをトレーニングしたい場合、[コード](https://github.com/tensorflow/magenta/tree/master/magenta/models/music_vae)と[チェックポイント](http://download.magenta.tensorflow.org/models/music_vae/checkpoints.tar.gz)を別々にダウンロードしてローカルで実行します。



## 概要説明

1. 非表示のセルをダブルクリックして表示するか、上部のメニューで[表示]&gt; [セクションを展開]を選択します。 
1. 各セルの左上隅にある`[ ]`カーソルを合わせ、 `再生`ボタンをクリックして順番に実行します。 
1. 生成されたサンプルを聴きます。 
1. ノートブックをコピーしたり、コードを修正したり、自分のモデルを訓練したり、自分のMIDIをアップロードしたりするなど、自分だけのものにしましょう。 



## A. 環境を準備する

シーケンス合成用のパッケージのインストールが含まれています。数分かかります。 


In [1]:
#added on 2020/6/20
%tensorflow_version 1.x

#@title Setup Environment
#@test {"output": "ignore"}

import glob

print 'Copying checkpoints and example MIDI from GCS. This will take a few minutes...'
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/* /content/

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -q magenta==0.5

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print 'Importing libraries and defining some helper functions...'
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow as tf

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print 'Start Seq Reconstruction'
  play(note_sequences[0])
  print 'End Seq Reconstruction'
  play(note_sequences[-1])
  print 'Mean Sequence'
  play(note_sequences[num_steps // 2])
  print 'Start -> End Interpolation'
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print 'Done'

Copying checkpoints and example MIDI from GCS. This will take a few minutes...
Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 110845 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
apache-beam 2.9.0 has requirement pytz<=2018.4,>=2018.3, but you'll have pytz 2018.7 which is incompatible.
Importing libraries and defining some helper functions...


/usr/local/lib/python2.7/dist-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._max_len_seq_inner import _max_len_seq_inner
/usr/local/lib/python2.7/dist-packages/scipy/signal/_upfirdn.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._upfirdn_apply import _output_len, _apply
/usr/local/lib/python2.7/dist-packages/scipy/signal/spectral.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._spectral import _lombscargle
/usr/local/lib/python2.7/dist-packages/scipy/signal/_peak_finding.py:13: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._peak_finding_utils import (_argmaxima1d, _select_by_peak_distance,
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/base.py:35: RuntimeWarning: numpy.dtype size ch

Done


/usr/local/lib/python2.7/dist-packages/sklearn/isotonic.py:11: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._isotonic import _inplace_contiguous_isotonic_regression, _make_unique
/usr/local/lib/python2.7/dist-packages/sklearn/manifold/t_sne.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _utils
/usr/local/lib/python2.7/dist-packages/sklearn/manifold/t_sne.py:27: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _barnes_hut_tsne
/usr/local/lib/python2.7/dist-packages/sklearn/manifold/t_sne.py:27: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _barnes_hut_tsne
/usr/local/lib/python2.7/dist-packages/sklearn/tree/tree.py:40: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  f


## B. データセットを準備する

## C. データセットを前処理する

モデルは学習済みで、予め準備したプライマーを元に演奏を生成するため、データセットの準備は不要です。

ここから先は、生成する音楽のスタイルごとに、学習済みモデルの取得・生成の試行をそれぞれ試します。

- 2小節ドラムモデル
- 2小節メロディモデル
- 16小節メロディモデル
- 16小節トリオモデル



# 2小節ドラムモデル

以下は、実験する4つの事前トレーニング済みモデルです。最初の3つは、61のMIDIドラム「ピッチ」を、9つのクラス（ベース、スネア、クローズドハイハット、オープンハイハット、ロータム、ミッドタム、ハイタム、クラッシュシンバル、ライドシンバル）という簡略化し、表現力を落とした出力空間にマッピングします。最後のモデルは、すべての可能なMIDIドラム「ピッチ」を表すために[NADE](http://homepages.inf.ed.ac.uk/imurray2/pub/11nade/)を使用します。 
-  **drums_2bar_oh_lokl** ：この*低* KLモデルは、より*リアルな*サンプリングのために学習されました。出力は、ドラムヒットの2 ^ 9の組み合わせのワンホットエンコーディングです。各方向に512ノードの単層双方向LSTMエンコーダ、各層に256ノードの2層LSTMデコーダ、および256次元のZがあります。トレーニング中に0の空きビットが与えられ、0.8の固定ベータ値がありました。 300kステップ後の最終精度は0.73、KLダイバージェンスは11ビットです。 
-  **drums_2bar_oh_hikl** ：この*高* KLモデルは、 *再構築と補間の改善*のために学習されました。出力は、ドラムヒットの2 ^ 9の組み合わせのワンホットエンコーディングです。各方向に512ノードの単層双方向LSTMエンコーダ、各層に256ノードの2層LSTMデコーダ、および256次元のZがあります。トレーニング中に96個の空きビットが与えられ、ベータ値は0.2に固定されていました。それは逆シグモイドスケジュールとレート1000でスケジュールされたサンプリングで学習しました。300kの後、ステップの最終的な精度は0.97で、KLダイバージェンスは107ビットです。 
-  **drums_2bar_nade_reduced** ：このモデルは9クラスのマルチラベル "pianoroll"を出力します。各方向に512のノードを持つ単層双方向LSTMエンコーダ、各層に512のノードを持つ2層のLSTM-NADEデコーダ、128の隠れユニットを持つ9次元のNADE、そして256の次元を持つZがあります。トレーニング中に96の空きビットが与えられ、0.2の固定ベータ値があります。それは逆シグモイドスケジュールとレート1000でスケジュールされたサンプリングで学習しました。300kステップの後、最終的な精度は0.98で、KLダイバージェンスは110ビットです。 
-  **drums_2bar_nade_full** ：出力は61クラスのマルチラベル "pianoroll"です。各方向に512個のノードを持つ単層双方向LSTMエンコーダ、各層に512個のノードを持つ2層LSTM-NADEデコーダ、および128個の隠れユニットを持つ61次元NADE、そして256次元のZ。トレーニング中に0の空きビットが与えられ、0.2の固定ベータ値があります。それは逆シグモイドスケジュールとレート1000でスケジュールされたサンプリングで学習しました。300kステップの後、最終的な精度は0.90であり、KLダイバージェンスは116ビットです。 


## D. 学習済みモデルを取得する

In [ ]:
#@title Load Pretrained Models

drums_models = {}
# One-hot encoded.
drums_config = configs.CONFIG_MAP['cat-drums_2bar_small']
drums_models['drums_2bar_oh_lokl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/drums_2bar_small.lokl.ckpt')
drums_models['drums_2bar_oh_hikl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/drums_2bar_small.hikl.ckpt')

# Multi-label NADE.
drums_nade_reduced_config = configs.CONFIG_MAP['nade-drums_2bar_reduced']
drums_models['drums_2bar_nade_reduced'] = TrainedModel(drums_nade_reduced_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/drums_2bar_nade.reduced.ckpt')
drums_nade_full_config = configs.CONFIG_MAP['nade-drums_2bar_full']
drums_models['drums_2bar_nade_full'] = TrainedModel(drums_nade_full_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/drums_2bar_nade.full.ckpt')


## E. モデルを学習する

学習済みモデルを使うため不要です。


## F. 学習済みモデルを評価する


### 1. サンプルを生成する

In [ ]:
#@title Generate 4 samples from the prior of one of the models listed above.
drums_sample_model = "drums_2bar_oh_lokl" #@param ["drums_2bar_oh_lokl", "drums_2bar_oh_hikl", "drums_2bar_nade_reduced", "drums_2bar_nade_full"]
temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
drums_samples = drums_models[drums_sample_model].sample(n=4, length=32, temperature=temperature)
for ns in drums_samples:
  play(ns)

In [ ]:
#@title Optionally download generated MIDI samples.
for i, ns in enumerate(drums_samples):
  download(ns, '%s_sample_%d.mid' % (drums_sample_model, i))


### 2. 補間を生成する


In [ ]:
#@title Option 1: Use example MIDI files for interpolation endpoints.
input_drums_midi_data = [
    tf.gfile.Open(fn).read()
    for fn in sorted(tf.gfile.Glob('/content/midi/drums_2bar*.mid'))]

In [ ]:
#@title Option 2: upload your own MIDI files to use for interpolation endpoints instead of those provided.
input_drums_midi_data = files.upload().values() or input_drums_midi_data

In [ ]:
#@title Extract drums from MIDI files. This will extract all unique 2-bar drum beats using a sliding window with a stride of 1 bar.
drums_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_drums_midi_data]
extracted_beats = []
for ns in drums_input_seqs:
  extracted_beats.extend(drums_nade_full_config.data_converter.to_notesequences(
      drums_nade_full_config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_beats):
  print "Beat", i
  play(ns)

In [ ]:
#@title Interpolate between 2 beats, selected from those in the previous cell.
drums_interp_model = "drums_2bar_oh_hikl" #@param ["drums_2bar_oh_lokl", "drums_2bar_oh_hikl", "drums_2bar_nade_reduced", "drums_2bar_nade_full"]
start_beat = 0 #@param {type:"integer"}
end_beat = 1 #@param {type:"integer"}
start_beat = extracted_beats[start_beat]
end_beat = extracted_beats[end_beat]

temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
num_steps = 13 #@param {type:"integer"}

drums_interp = interpolate(drums_models[drums_interp_model], start_beat, end_beat, num_steps=num_steps, temperature=temperature)

In [ ]:
#@title Optionally download interpolation MIDI file.
download(drums_interp, '%s_interp.mid' % drums_interp_model)


# 2小節メロディモデル

事前学習済みモデルは、各方向に2048ノードの単層双方向LSTMエンコーダ、各層に2048ノードの3層LSTMデコーダ、および512次元のZから構成されています。このモデルには0フリービットが与えられ、そのベータ値は200kステップにわたって0から0.43まで0.99999の指数関数的速度でアニーリングされました。逆シグモイドスケジュールとレート1000でスケジュールされたサンプリングで学習されました。最終的な精度は0.95であり、KLダイバージェンスは58ビットです。 


## D. 学習済みモデルを取得する

In [ ]:
#@title Load the pre-trained model.
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/mel_2bar_big.ckpt')

## E. モデルを学習する

学習済みモデルを使うため不要です。


## F. 学習済みモデルを評価する


### 1. サンプルを生成する


In [ ]:
#@title Generate 4 samples from the prior.
temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
mel_2_samples = mel_2bar.sample(n=4, length=32, temperature=temperature)
for ns in mel_2_samples:
  play(ns)

In [ ]:
#@title Optionally download samples.
for i, ns in enumerate(mel_2_samples):
  download(ns, 'mel_2bar_sample_%d.mid' % i)


### 2. 補間を生成する


In [ ]:
#@title Option 1: Use example MIDI files for interpolation endpoints.
input_mel_midi_data = [
    tf.gfile.Open(fn).read()
    for fn in sorted(tf.gfile.Glob('/content/midi/mel_2bar*.mid'))]

In [ ]:
#@title Option 2: Upload your own MIDI files to use for interpolation endpoints instead of those provided.
input_mel_midi_data = files.upload().values() or input_mel_midi_data

In [ ]:
#@title Extract melodies from MIDI files. This will extract all unique 2-bar melodies using a sliding window with a stride of 1 bar.
mel_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_mel_midi_data]
extracted_mels = []
for ns in mel_input_seqs:
  extracted_mels.extend(
      mel_2bar_config.data_converter.to_notesequences(
          mel_2bar_config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_mels):
  print "Melody", i
  play(ns)

In [ ]:
#@title Interpolate between 2 melodies, selected from those in the previous cell.
start_melody = 0 #@param {type:"integer"}
end_melody = 1 #@param {type:"integer"}
start_mel = extracted_mels[start_melody]
end_mel = extracted_mels[end_melody]

temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
num_steps = 13 #@param {type:"integer"}

mel_2bar_interp = interpolate(mel_2bar, start_mel, end_mel, num_steps=num_steps, temperature=temperature)

In [ ]:
#@title Optionally download interpolation MIDI file.
download(mel_2bar_interp, 'mel_2bar_interp.mid')


# 16小節メロディモデル

事前学習済み階層モデルは、各層各方向に2048ノードを持つ2層双方向LSTMエンコーダ、各層に1024ノードを持つ16ステップ2層LSTM「指揮者」デコーダ、2層LSTM各レイヤに1024個のノードを持つコアデコーダ、および512次元のZから構成されています。256の空きビットが与えられ、ベータ値は0.2に固定されていました。 25kステップ後の最終精度は0.90で、KLダイバージェンスは277ビットです。 


## D. 学習済みモデルを取得する

In [ ]:
#@title Load the pre-trained models.
mel_16bar_models = {}
hierdec_mel_16bar_config = configs.CONFIG_MAP['hierdec-mel_16bar']
mel_16bar_models['hierdec_mel_16bar'] = TrainedModel(hierdec_mel_16bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/mel_16bar_hierdec.ckpt')

flat_mel_16bar_config = configs.CONFIG_MAP['flat-mel_16bar']
mel_16bar_models['baseline_flat_mel_16bar'] = TrainedModel(flat_mel_16bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/mel_16bar_flat.ckpt')

## E. モデルを学習する

学習済みモデルを使うため不要です。

## F. 学習済みモデルを評価する


### 1. サンプルを生成する


In [ ]:
#@title Generate 4 samples from the selected model prior.
mel_sample_model = "hierdec_mel_16bar" #@param ["hierdec_mel_16bar", "baseline_flat_mel_16bar"]
temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
mel_16_samples = mel_16bar_models[mel_sample_model].sample(n=4, length=256, temperature=temperature)
for ns in mel_16_samples:
  play(ns)

In [ ]:
#@title Optionally download MIDI samples.
for i, ns in enumerate(mel_16_samples):
  download(ns, '%s_sample_%d.mid' % (mel_sample_model, i))


### 2. 補間を生成する


In [ ]:
#@title Option 1: Use example MIDI files for interpolation endpoints.
input_mel_16_midi_data = [
    tf.gfile.Open(fn).read()
    for fn in sorted(tf.gfile.Glob('/content/midi/mel_16bar*.mid'))]

In [ ]:
#@title Option 2: upload your own MIDI files to use for interpolation endpoints instead of those provided.
input_mel_16_midi_data = files.upload().values() or input_mel_16_midi_data

In [ ]:
#@title Extract melodies from MIDI files. This will extract all unique 16-bar melodies using a sliding window with a stride of 1 bar.
mel_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_mel_16_midi_data]
extracted_16_mels = []
for ns in mel_input_seqs:
  extracted_16_mels.extend(
      hierdec_mel_16bar_config.data_converter.to_notesequences(
          hierdec_mel_16bar_config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_16_mels):
  print "Melody", i
  play(ns)

In [ ]:
#@title Compute the reconstructions and mean of the two melodies, selected from the previous cell.
mel_interp_model = "hierdec_mel_16bar" #@param ["hierdec_mel_16bar", "baseline_flat_mel_16bar"]

start_melody = 0 #@param {type:"integer"}
end_melody = 1 #@param {type:"integer"}
start_mel = extracted_16_mels[start_melody]
end_mel = extracted_16_mels[end_melody]

temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}

mel_16bar_mean = interpolate(mel_16bar_models[mel_interp_model], start_mel, end_mel, num_steps=3, max_length=256, individual_duration=32, temperature=temperature)

In [ ]:
#@title Optionally download mean MIDI file.
download(mel_16bar_mean, '%s_mean.mid' % mel_interp_model)


# 16小節「トリオ」モデル（リード、ベース、ドラム） 

 16小節トリオの2つの事前学習モデルを提示します。階層モデルとフラット（ベースライン）モデルです。 

事前学習済み階層モデルは、各層両方向に2048ノードを持つ2層スタックの双方向LSTMエンコーダ、各層に1024ノードを持つ16ステップ2層LSTMの「指揮者」デコーダ、各層の1024ノードを持つ3つ（リード、ベース、ドラム）の2層LSTMコアデコーダ、および512次元のZから構成されています。 1024の空きビットが与えられ、固定ベータ値は0.1でした。それは逆シグモイドスケジュールとレート1000で、スケジュールされたサンプリングで学習しました。50kステップの後、最終的な精度はリード 0.82、ベース 0.87、ドラム 0.90、そしてKLダイバージェンスは1027ビットです。 

事前学習済みフラットモデルは、各層両方向に2048ノードを持つ2層スタックの双方向LSTMエンコーダ、各層に2048ノードを持つ3層のLSTMデコーダ、および512次元のZから構成されています。 1024の空きビットが与えられ、固定ベータ値は0.1でした。それは逆シグモイドスケジュールとレート1000で、スケジュールされたサンプリングで学習しました。50kステップの後、最終的な精度はリード 0.67、ベース 0.66、ドラム 0.79、そしてKLダイバージェンスは1016ビットです。 


## D. 学習済みモデルを取得する

In [ ]:
#@title Load the pre-trained models.
trio_models = {}
hierdec_trio_16bar_config = configs.CONFIG_MAP['hierdec-trio_16bar']
trio_models['hierdec_trio_16bar'] = TrainedModel(hierdec_trio_16bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/trio_16bar_hierdec.ckpt')

flat_trio_16bar_config = configs.CONFIG_MAP['flat-trio_16bar']
trio_models['baseline_flat_trio_16bar'] = TrainedModel(flat_trio_16bar_config, batch_size=4, checkpoint_dir_or_path='/content/checkpoints/trio_16bar_flat.ckpt')

## E. モデルを学習する

学習済みモデルを使うため不要です。

## F. 学習済みモデルを評価する


### 1. サンプルを生成する


In [ ]:
#@title Generate 4 samples from the selected model prior.
trio_sample_model = "hierdec_trio_16bar" #@param ["hierdec_trio_16bar", "baseline_flat_trio_16bar"]
temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}

trio_16_samples = trio_models[trio_sample_model].sample(n=4, length=256, temperature=temperature)
for ns in trio_16_samples:
  play(ns)

In [ ]:
#@title Optionally download MIDI samples.
for i, ns in enumerate(trio_16_samples):
  download(ns, '%s_sample_%d.mid' % (trio_sample_model, i))


### 2. 補間を生成する


In [ ]:
#@title Option 1: Use example MIDI files for interpolation endpoints.
input_trio_midi_data = [
    tf.gfile.Open(fn).read()
    for fn in sorted(tf.gfile.Glob('/content/midi/trio_16bar*.mid'))]

In [ ]:
#@title Option 2: Upload your own MIDI files to use for interpolation endpoints instead of those provided.
input_trio_midi_data = files.upload().values() or input_trio_midi_data

In [ ]:
#@title Extract trios from MIDI files. This will extract all unique 16-bar trios using a sliding window with a stride of 1 bar.
trio_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_trio_midi_data]
extracted_trios = []
for ns in trio_input_seqs:
  extracted_trios.extend(
      hierdec_trio_16bar_config.data_converter.to_notesequences(
          hierdec_trio_16bar_config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_trios):
  print "Trio", i
  play(ns)

In [ ]:
#@title Compute the reconstructions and mean of the two trios, selected from the previous cell.
trio_interp_model = "hierdec_trio_16bar" #@param ["hierdec_trio_16bar", "baseline_flat_trio_16bar"]

start_trio = 0 #@param {type:"integer"}
end_trio = 1 #@param {type:"integer"}
start_trio = extracted_trios[start_trio]
end_trio = extracted_trios[end_trio]

temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
trio_16bar_mean = interpolate(trio_models[trio_interp_model], start_trio, end_trio, num_steps=3, max_length=256, individual_duration=32, temperature=temperature)

In [ ]:
#@title Optionally download mean MIDI file.
download(trio_16bar_mean, '%s_mean.mid' % trio_interp_model)